In [3]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# executable_path = {'executable_path': '\bin\chromedriver.exe'}
# browser = Browser('chrome')

# Scrape NASA Mars News

In [5]:
# URL of page to be scraped

url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html

In [6]:
# Create BeautifulSoup object; parse with 'html.parser'

soup = BeautifulSoup(html, 'html.parser')
mars_news = soup.find('li', class_='slide')

In [7]:
# Examine the results, then determine element that contains sought info

print(mars_news.prettify)

<bound method Tag.prettify of <li class="slide"><div class="image_and_description_container"><a href="/news/8654/how-nasas-perseverance-mars-team-adjusted-to-work-in-the-time-of-coronavirus/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="Members of NASA's Perseverance rover mission work remotely from home during the coronavirus outbreak." src="/system/news_items/list_view_images/8654_MAIN-IMAGE-PIA23881-320x240.jpg"/></div><div class="bottom_gradient"><div><h3>How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </h3></div></div></a><div class="list_text"><div class="list_date">April 21, 2020</div><div class="conte

In [8]:
news_title = mars_news.find('div', class_="content_title").text
news_p = mars_news.find('div', class_="article_teaser_body").text
print(news_title)
print(news_p)

How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus 
Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.


# Scrape JPL Mars Space Images

In [9]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Click on "Full Image"
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

# Click on "More Info"
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [10]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
jpl_soup = BeautifulSoup(html, 'html.parser')

jpl_image = jpl_soup.select_one('figure.lede a img').get('src')
print(jpl_image)

/spaceimages/images/largesize/PIA18826_hires.jpg


In [11]:
# Append previous result to main url
featured_image_url = f'https://www.jpl.nasa.gov{jpl_image}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18826_hires.jpg


# Scrape Mars Weather

In [12]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html

In [13]:
# Request information from Mars' twitter page
twitter_response = requests.get('https://twitter.com/marswxreport?lang=en')

# Create BeautifulSoup object; parse with 'html.parser'
twitter_soup = BeautifulSoup(twitter_response.text, "html.parser")

# Parent container for the top weather tweet
tweet_containers = twitter_soup.find_all("div", class_='js-tweet-text-container')

# Parse and print most recent weather tweet
mars_weather = tweet_containers[0].text
print(mars_weather)


InSight sol 501 (2020-04-24) low -93.5ºC (-136.2ºF) high -4.3ºC (24.3ºF)
winds from the SW at 5.0 m/s (11.3 mph) gusting to 15.8 m/s (35.3 mph)
pressure at 6.70 hPapic.twitter.com/Xdbw8T0T0E



# Scrape Mars Facts

In [14]:
# Visit the Mars Facts webpage
url = 'https://space-facts.com/mars/'

# Read the tables from the webpage
facts_tables = pd.read_html(url)
facts_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [15]:
# Select first table
facts_table_df = facts_tables[0]
facts_table_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
# Rename dataframe columns
facts_table_df.columns = ["Mars Facts", "Values"]

# Reset the index
facts_table_df.set_index("Mars Facts", inplace=True)
facts_table_df

,Values
Mars Facts,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [18]:
# Convert the dataframe to html table and replace line breaks with blanks

mars_facts = (facts_table_df.to_html()).replace('\n', '')
mars_facts

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Mars Facts</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Scrape Mars Hemispheres

In [20]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [21]:
# Create empty list for url's
hemisphere_img_urls = []

# Finding elements in the page, by selector type: css
links = browser.find_by_css('a.product-item h3')

# Loop through number of links to store titles and url's
for i in range(len(links)):
    
    # Create empty dictionary for titles and url's
    hemisphere = {}
    
    # Finding elements in the page, by selector type: css
    browser.find_by_css('a.product-item h3')[i].click()
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['title'] = browser.find_by_css('h2.title').text
    hemisphere['img_url']  = sample_elem['href']
    hemisphere_img_urls.append(hemisphere)
    
    # loads the previous URL in the history list (back button)
    browser.back() 

In [22]:
# Show list of dictionaries containing titles and url's
hemisphere_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [25]:
hemisphere_img_urls[0]['title']

'Cerberus Hemisphere Enhanced'

In [26]:
hemisphere_img_urls[0]['img_url']

'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'